<a href="https://colab.research.google.com/github/beta996/Twitter-Sentiment-Analysis/blob/main/MLproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [116]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.stem.snowball import SnowballStemmer

In [6]:
%load_ext google.colab.data_table

In [7]:
df1 = pd.read_csv('gdrive/MyDrive/ML_project/Twitter_Data.csv')

In [34]:
df1.shape

(162980, 2)

In [35]:
df1.isnull().sum()

text        4
category    7
dtype: int64

In [36]:
df1.dropna(axis=0, inplace=True)

In [38]:
df1 = df1.rename(columns={'clean_text':'text'})
df1['category'] = df1['category'].astype(int)

In [39]:
df1.head()

,text,category
0,when modi promised “minimum government maximum...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


In [10]:
df2 = pd.read_csv('gdrive/MyDrive/ML_project/apple-twitter-sentiment-texts.csv')

In [14]:
df2 = df2.rename(columns={'sentiment':'category'})

In [41]:
df2.isnull().sum()

text        0
category    0
dtype: int64

In [16]:
df2.head()

,text,category
0,Wow. Yall needa step it up @Apple RT @heynyla:...,-1
1,What Happened To Apple Inc? http://t.co/FJEX...,0
2,Thank u @apple I can now compile all of the pi...,1
3,The oddly uplifting story of the Apple co-foun...,0
4,@apple can i exchange my iphone for a differen...,0


In [18]:
df3 = pd.read_csv('gdrive/MyDrive/ML_project/Tweets.csv')

In [ ]:
df3.head()

In [ ]:
df3 = df3[['text', 'airline_sentiment']]

In [26]:
df3 = df3.rename(columns={'airline_sentiment':'category'})
df3['category'] = df3['category'].map({'negative': -1, 'neutral': 0, 'positive':1})

In [42]:
df3.isnull().sum()

text        0
category    0
dtype: int64

In [27]:
df3.head()

,text,category
0,@VirginAmerica What @dhepburn said.,0
1,@VirginAmerica plus you've added commercials t...,1
2,@VirginAmerica I didn't today... Must mean I n...,0
3,@VirginAmerica it's really aggressive to blast...,-1
4,@VirginAmerica and it's a really big bad thing...,-1


In [141]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df_oryginal = df.copy()

In [132]:
df.shape

(179239, 2)

In [59]:
df['category'].value_counts()

 1    74755
 0    59111
-1    45373
Name: category, dtype: int64

In [ ]:
# there are hyperlinks
[itm[0] for itm in df['text'].str.findall('http?:\/\/[^\s]+|www\.[^\s]') if len(itm)>0]

In [ ]:
# there are hashtags
[itm[0] for itm in df['text'].str.findall('#\w+') if len(itm)>0]

In [ ]:
# there are users
[itm[0] for itm in df['text'].str.findall('@[\w]+') if len(itm)>0]

In [ ]:
# there are numbers
[itm[0] for itm in df['text'].str.findall('.*\d+.*') if len(itm)>0]

In [133]:
def preprocess(df):
  special_chars = ['~', ':', "'", '+', '[', '\\', '@', '^', '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '!', '>', ';', '?', '#', '$', ')', '/']

  df['text'] = df['text'].str.lower() # convert to lowercase
  df['text'] = df['text'].str.replace('http?:\/\/[^\s]+|www\.[^\s]', '', regex=True) # remove hyperlinks
  df['text'] = df['text'].str.replace('@[\w]+', '', regex=True) # remove users
  df['text'] = df['text'].str.replace('#\w+', '', regex=True) # remove hashtags
  df['text'] = df['text'].str.replace('\d+', '', regex=True) # remove numbers

  for char in special_chars:
    df['text'] = df['text'].str.replace(char, '') # special characters

  # remove stopwords
  nltk.download('stopwords')
  stop_words = []
  with open('/root/nltk_data/corpora/stopwords/english','r') as file:
    stop_words = [word for word in file.read().splitlines()]
  df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  df = stemming(df)
  return df

In [122]:
def stemming(df:pd.DataFrame):
  stemmer = SnowballStemmer("english")
  unstemmed = df['text'].str.split() # tokenizing
  stemmed = unstemmed.apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word
  for i in range(len(stemmed)):
    stemmed[i] = ' '.join(stemmed[i])
  df['text'] = stemmed
  return df

In [135]:
preprocessed = preprocess(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [142]:
compare = pd.DataFrame
compare = preprocessed.tail()
compare['before'] = df_oryginal['text']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,category,before
179234,thank got differ flight chicago,1,@AmericanAir thank you we got on a different f...
179235,leav minut late flight warn communic minut lat...,-1,@AmericanAir leaving over 20 minutes Late Flig...
179236,plea bring american airlin,0,@AmericanAir Please bring American Airlines to...
179237,money chang flight dont answer phone suggest m...,-1,"@AmericanAir you have my money, you change my ..."
179238,ppl need know mani seat next flight plz put us...,0,@AmericanAir we have 8 ppl so we need 2 know h...
